In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
nn_model_id = 'retrained_sherlock'

In [3]:
import pandas as pd
import numpy as np

from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock
from datetime import datetime
from sklearn.metrics import f1_score, classification_report

### Load training and validation sets

In [4]:
start = datetime.now()
print(f'Started at {start}')

X_train = pd.read_parquet('train.parquet')
y_train = pd.read_parquet('../data/raw/train_labels.parquet').values.flatten()

y_train = np.array([x.lower() for x in y_train])

print(f'Load data (train) process took {datetime.now() - start} seconds.')

Started at 2022-02-04 11:53:09.566836
Load data (train) process took 0:00:03.543899 seconds.


In [5]:
print('Distinct types for columns in the Dataframe (should be all float32):')
print(set(X_train.dtypes))

Distinct types for columns in the Dataframe (should be all float32):
{dtype('float32')}


In [6]:
start = datetime.now()
print(f'Started at {start}')

X_validation = pd.read_parquet('validation.parquet')
y_validation = pd.read_parquet('../data/raw/val_labels.parquet').values.flatten()

y_validation = np.array([x.lower() for x in y_validation])

print(f'Load data (validation) process took {datetime.now() - start} seconds.')

Started at 2022-02-04 11:53:13.378367
Load data (validation) process took 0:00:01.639247 seconds.


### Re-train Sherlock

In [7]:
start = datetime.now()
print(f'Started at {start}')

train_sherlock(X_train, y_train, X_validation, y_validation, nn_id=nn_model_id);

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-02-04 11:53:15.081434


W0204 11:53:15.555608 4443889152 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0204 11:53:15.556807 4443889152 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0204 11:53:15.561105 4443889152 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/s

Successfully loaded and compiled model, now fitting model on data.
Train on 412059 samples, validate on 137353 samples


2022-02-04 11:53:18.849842: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-04 11:53:18.880937: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd57ebc7030 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-04 11:53:18.880972: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Epoch 1/100
412059/412059 [==============================] - 80s 193us/sample - loss: 1.6102 - categorical_accuracy: 0.6984 - val_loss: 1.0352 - val_categorical_accuracy: 0.8294
Epoch 2/100
412059/412059 [==============================] - 84s 204us/sample - loss: 0.9596 - categorical_accuracy: 0.8368 - val_loss: 0.9416 - val_categorical_accuracy: 0.8499
Epoch 3/100
412059/412059 [==============================] - 78s 189us/sample - loss: 0.8494 - categorical_accuracy: 0.8597 - val_loss: 0.8910 - val_categorical_accuracy: 0.8578
Epoch 4/100
412059/412059 [==============================] - 80s 194us/sample - loss: 0.7843 - categorical_accuracy: 0.8711 - val_loss: 0.8555 - val_categorical_accuracy: 0.8624
Epoch 5/100
412059/412059 [==============================] - 79s 192us/sample - loss: 0.7363 - categorical_accuracy: 0.8798 - val_loss: 0.8280 - val_categorical_accuracy: 0.8667
Epoch 6/100
412059/412059 [==============================] - 78s 190us/sample - loss: 0.6994 - categorical_acc

### Load test set

In [8]:
start = datetime.now()
print(f'Started at {start}')

X_test = pd.read_parquet('test.parquet')
y_test = pd.read_parquet('../data/raw/test_labels.parquet').values.flatten()

y_test = np.array([x.lower() for x in y_test])

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-02-04 12:48:46.477211
Trained and saved new model.
Finished at 2022-02-04 12:48:48.335172, took 0:00:01.857975 seconds


### Make prediction

In [9]:
predicted_labels = predict_sherlock(X_test, nn_id=nn_model_id)

In [10]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

prediction count 137353, type = <class 'numpy.ndarray'>


0.8936850535436299

In [11]:
# Best results
# 0.8956975031011328, adam optimiser, took 0:59:19.320310 seconds (retrained_sherlock8)

# 0.8871638630500646, SGD Nesterov optimiser,  took 1:37:09.679227 seconds (retrained_sherlock9)

# LAST
# 0.8941992181577104
# 0.8661276466591463
# 0.8765360861865846
# 0.8777825108666634
# 0.8709458033528152
# 0.8526528126158138
# 0.836800124917518
# 0.8590214693450329
# 0.8583626471084315

In [12]:
classes = np.load(f"../sherlock/deploy/classes_{nn_model_id}.npy", allow_pickle=True)

report = classification_report(y_test, predicted_labels, output_dict=True)

class_scores = list(filter(lambda x: isinstance(x, tuple) and isinstance(x[1], dict) and 'f1-score' in x[1] and x[0] in classes, list(report.items())))

class_scores = sorted(class_scores, key=lambda item: item[1]['f1-score'], reverse=True)

### Top 5 Types

In [13]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[0:5]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
isbn		0.992		0.997		0.987		1430
grades		0.992		0.990		0.993		1765
jockey		0.987		0.987		0.987		2819
industry	0.982		0.976		0.987		2958
currency	0.978		0.987		0.968		405


### Bottom 5 Types

In [14]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[len(class_scores)-5:len(class_scores)]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
rank		0.709		0.671		0.751		2983
person		0.663		0.729		0.608		579
director	0.569		0.621		0.524		225
sales		0.527		0.711		0.419		322
ranking		0.517		0.786		0.385		439


### All Scores

In [15]:
print(classification_report(y_test, predicted_labels, digits=3))

                precision    recall  f1-score   support

       address      0.936     0.938     0.937      3003
     affiliate      0.966     0.824     0.889       204
   affiliation      0.977     0.955     0.966      1768
           age      0.863     0.953     0.905      3033
         album      0.889     0.886     0.887      3035
          area      0.887     0.819     0.852      1987
        artist      0.822     0.868     0.844      3043
    birth date      0.979     0.975     0.977       479
   birth place      0.977     0.909     0.942       418
         brand      0.751     0.716     0.733       574
      capacity      0.826     0.693     0.754       362
      category      0.885     0.901     0.893      3087
          city      0.836     0.908     0.871      2966
         class      0.878     0.918     0.897      2971
classification      0.935     0.859     0.895       587
          club      0.975     0.955     0.965      2977
          code      0.929     0.904     0.916  

## Review errors

In [16]:
from collections import Counter

size = len(y_test)
mismatches = list()

for idx, k1 in enumerate(y_test[:size]):
    k2 = predicted_labels[idx]

    if k1 != k2:
        mismatches.append(k1)
        
        # zoom in to specific errors. Use the index in the next step
        if k1 in ('address'):
            print(f'[{idx}] expected "{k1}" but predicted "{k2}"')
        
f1 = f1_score(y_test[:size], predicted_labels[:size], average="weighted")
print(f'Total mismatches: {len(mismatches)} (F1 score: {f1})')

data = Counter(mismatches)
data.most_common()   # Returns all unique items and their counts

[1116] expected "address" but predicted "name"
[2420] expected "address" but predicted "location"
[3398] expected "address" but predicted "city"
[4354] expected "address" but predicted "location"
[5112] expected "address" but predicted "location"
[5647] expected "address" but predicted "team name"
[6526] expected "address" but predicted "location"
[7119] expected "address" but predicted "city"
[8797] expected "address" but predicted "area"
[9354] expected "address" but predicted "location"
[9574] expected "address" but predicted "location"
[9806] expected "address" but predicted "city"
[10035] expected "address" but predicted "creator"
[10067] expected "address" but predicted "region"
[11055] expected "address" but predicted "city"
[11902] expected "address" but predicted "name"
[11993] expected "address" but predicted "location"
[12072] expected "address" but predicted "artist"
[13221] expected "address" but predicted "status"
[14677] expected "address" but predicted "location"
[15232

Total mismatches: 14568 (F1 score: 0.8936850535436299)


[('name', 745),
 ('rank', 744),
 ('position', 555),
 ('region', 502),
 ('location', 500),
 ('description', 479),
 ('artist', 401),
 ('team', 387),
 ('notes', 379),
 ('area', 359),
 ('album', 345),
 ('type', 343),
 ('company', 332),
 ('product', 322),
 ('category', 305),
 ('code', 284),
 ('gender', 276),
 ('team name', 274),
 ('city', 272),
 ('ranking', 270),
 ('day', 258),
 ('class', 245),
 ('weight', 232),
 ('person', 227),
 ('status', 213),
 ('owner', 211),
 ('credit', 200),
 ('country', 196),
 ('year', 194),
 ('sales', 187),
 ('address', 185),
 ('sex', 178),
 ('result', 177),
 ('origin', 173),
 ('service', 169),
 ('duration', 166),
 ('manufacturer', 164),
 ('brand', 163),
 ('age', 144),
 ('component', 142),
 ('state', 141),
 ('county', 141),
 ('range', 139),
 ('plays', 136),
 ('club', 133),
 ('format', 123),
 ('order', 122),
 ('capacity', 111),
 ('director', 107),
 ('symbol', 94),
 ('nationality', 93),
 ('publisher', 88),
 ('command', 85),
 ('classification', 83),
 ('affiliation', 8

In [17]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')

In [18]:
from ast import literal_eval

idx = 1001
original = test_samples.iloc[idx]
converted = original.apply(literal_eval).to_list()

print(f'Predicted "{predicted_labels[idx]}", actual label "{y_test[idx]}". Actual values:\n{converted}')

Predicted "address", actual label "address". Actual values:
[['Cabot House', 'Cabot House', '5 Hill Rd.', '5 Hill Rd.', '9 Cabot Rd.', '9 Cabot Rd.', 'Cabot House', '22 Bank Rd.', '22 Bank Rd.', 'Cabot House', '31 Bank Rd.', '31 Bank Rd.', 'Bairds Hotel', '11 Cabot Rd.', '11 Cabot Rd.', '10 Hill Rd.', '10 Hill Rd.', '10 Hill Rd.', '10 Hill Rd.', '7A Church Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '2 Coronation St.', '2 Coronation St.', '7A Church Rd.', '12 Hill Rd.', '12 Hill Rd.', '12 Hill Rd.', 'Cabot House', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '7A Church Rd.', '18 Mill Rd.', '17 Hill Rd.', '17 Hill Rd.', 'Cabot House', 'Cabot House', '25 Bank Rd.', '10 Coronation St.', '6 Cabot Rd.', '6 Cabot Rd.', '8 Hill Rd.', '8 Hill Rd.', '4 Mill Rd.', '4 Mill Rd.', '12 Sulva Rd.', '4 Haig Rd.', '13 Botwood Rd.', '13 Botwood Rd.', '8 Botwood Rd.', '8 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd.